In [15]:
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
import string

In [2]:
train = pd.read_csv('/home/joaquin/Desktop/Datos/tp2/train.csv')
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
#Separo el label del set
X,y = train.iloc[:,:-1],train.iloc[:,-1]
X.head()

,id,keyword,location,text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...


In [8]:
#creo features
X['character_count'] = X['text'].str.len()
X['mean_word_length'] = X['text'].apply(lambda x: (sum(len(w) for w in str(x).split()) / len(str(x).split())))
X['punctuation_count'] = X['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
X = X.drop(['id','keyword','location','text'], axis = 1)
X.head()

,character_count,mean_word_length,punctuation_count
0,69,4.384615,1
1,38,4.571429,1
2,133,5.090909,3
3,65,7.125000,2
4,88,4.500000,2


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [18]:
#XGBoost
xg_bin = xgb.XGBClassifier(objective = 'binary:logistic',
                          colsample_btree = 0.3, learning_rate=0.1,
                          max_depth = 5, alpha = 10, n_estimators = 100)

In [19]:
#Entreno
xg_bin.fit(X_train,y_train)

[12:49:21] WARNING: /workspace/src/learner.cc:480: 
Parameters: { colsample_btree } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_btree=0.3,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=5, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, objective='binary:logistic', random_state=0,
              reg_alpha=10, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [20]:
preds = xg_bin.predict(X_test)
preds

array([1, 0, 0, ..., 0, 0, 0])

In [21]:
accuracy = accuracy_score(y_test,preds)
print('Accuracy: %.2f%%' % (accuracy * 100))

Accuracy: 66.84%
